### Лабораторная работа № 2

#### 1. Установка библиотек

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas
import numpy

#### 2. Загрузка датасета

In [18]:
df = pd.read_pickle('./data/clean_car_data.pkl')
df.info()
print(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_Name       299 non-null    object 
 1   Year           299 non-null    int64  
 2   Selling_Price  299 non-null    float64
 3   Present_Price  299 non-null    float64
 4   Driven_kms     299 non-null    int64  
 5   Fuel_Type      299 non-null    object 
 6   Selling_type   299 non-null    object 
 7   Transmission   299 non-null    object 
 8   Owner          299 non-null    int64  
 9   Car_Age        299 non-null    int64  
dtypes: float64(2), int64(4), object(4)
memory usage: 23.5+ KB
    Car_Name  Year  Selling_Price  Present_Price  Driven_kms Fuel_Type  \
0       ritz  2014           3.35           5.59       27000    Petrol   
1        sx4  2013           4.75           9.54       43000    Diesel   
2       ciaz  2017           7.25           9.85        6900    Petrol   
3    wag

#### 3. Переименовываем столбец целевой функции Selling_Price в Target и разделяем на тестовую и обучающую выборки

In [19]:
df = df.rename(columns={'Selling_Price': 'target'})
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.25, random_state=2)

#### 4. Определяем тип признаков object и number

In [20]:
cat_features = df.drop('target', axis=1) \
                 .select_dtypes(include=['object']).columns.to_list()

num_features = df.select_dtypes(include=['number']).drop(columns=['target']).columns.to_list()

print("Значения типа number:", num_features)
print("Значения типа object:", cat_features)

Значения типа number: ['Year', 'Present_Price', 'Driven_kms', 'Owner', 'Car_Age']
Значения типа object: ['Car_Name', 'Fuel_Type', 'Selling_type', 'Transmission']


#### 5. Создание модели и инициализация пайплайна

In [21]:
from sklearn.preprocessing import StandardScaler, TargetEncoder
from sklearn.ensemble import RandomForestRegressor
s_scaler = StandardScaler()
l_encoder = TargetEncoder()
regressor = RandomForestRegressor(random_state=42)

#### 6. Преобразования для числовых и категориальных признаков

In [22]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', s_scaler, num_features),  
        ('cat', l_encoder, cat_features), 
    ],
    remainder='drop' )

#### 7. Создание и обучение пайплайна

In [23]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline(steps=[('preprocessor', preprocessor), 
                           ('model', regressor)])

pipeline.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


#### 8. Оценка модели и расчет метрик средней абсолютной разницы между предсказаниями и реальными значениями, средней квадратичной разницы между предсказаниями и реальными значениями, а также средней абсолютной ошибки в процентах

In [24]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

predictions = pipeline.predict(X_test) 
metrics = {}
metrics["MAE"] = mean_absolute_error(y_test, predictions)
metrics["MSE"] = mean_squared_error(y_test, predictions)
metrics["MAPE"] = mean_absolute_percentage_error(y_test, predictions)

print(metrics)

{'MAE': 0.5486866666666669, 'MSE': 1.2086129750666665, 'MAPE': 0.21214137711842934}


#### 9. Импорт mlflow

In [25]:
import mlflow

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

registry_uri = f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}"
tracking_uri = f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}"

mlflow.set_tracking_uri(tracking_uri)   
mlflow.set_registry_uri(registry_uri)   

In [26]:
EXPERIMENT_NAME = "car_project"
RUN_NAME = "baseline model"
REGISTRY_MODEL_NAME = "car_price_prediction_model"

#### 10. Подготовка метаданных для модели

In [27]:
from mlflow.models import infer_signature

signature =  infer_signature(model_input = X_train.head(5))
input_example = X_train.head(5)

c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


In [28]:
req_file = './requirements.txt'
params_dict = pipeline.get_params()

#### 11. Запуск эксперимента в MLflow с random_state=42

In [29]:
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

if experiment is None:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id
    
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
  
    run_id = run.info.run_id 
    mlflow.sklearn.log_model(pipeline, 
                             artifact_path="models",
                             signature=signature,
                             input_example=input_example,
                             pip_requirements=req_file
                             )
    mlflow.log_metrics(metrics)

    mlflow.log_params(params_dict)

run = mlflow.get_run(run_id) 
assert (run.info.status =='FINISHED')

2025/11/05 19:13:20 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/05 19:13:25 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - numpy (current: 2.3.4, required: numpy==1.26.4)
 - mlflow (current: 3.5.1, required: mlflow==2.16)
 - seaborn (current: uninstalled, required: seaborn)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


2025/11/05 19:13:25 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - numpy (current: 2.3.4, required: numpy==1.26.4)
 - mlflow (current: 3.5.1, required: mlflow==2.16)
 - seaborn (current: uninstalled, required: seaborn)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


🏃 View run baseline model at: http://127.0.0.1:5000/#/experiments/818228273553268644/runs/3761c33eafdd4468b8b9adefac80de32
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/818228273553268644


#### 12. Запуск эксперимента в MLflow с random_state=43

In [30]:
regressor2 = RandomForestRegressor(random_state=43)

In [31]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor), 
                           ('model', regressor2)])

print(pipeline.fit(X_train, y_train))

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Year', 'Present_Price',
                                                   'Driven_kms', 'Owner',
                                                   'Car_Age']),
                                                 ('cat', TargetEncoder(),
                                                  ['Car_Name', 'Fuel_Type',
                                                   'Selling_type',
                                                   'Transmission'])])),
                ('model', RandomForestRegressor(random_state=43))])


#### 13.  Оценка модели и расчет метрик с random_state=43

In [32]:
predictions = pipeline.predict(X_test) 
metrics = {}
metrics["MAE"] = mean_absolute_error(y_test, predictions)
metrics["MSE"] = mean_squared_error(y_test, predictions)
metrics["MAPE"] = mean_absolute_percentage_error(y_test, predictions)

print(metrics)

{'MAE': 0.5715933333333337, 'MSE': 1.1843767988000011, 'MAPE': 0.21427208994643848}


#### 14. Сгенерировали новые признаки с помощью различных трансформаций из библиотеки sklearn

In [33]:
from sklearn.preprocessing import PolynomialFeatures, SplineTransformer, QuantileTransformer

X_train_fe_sklearn = X_train.copy()

pf = PolynomialFeatures(degree=2)

X_train_fe_sklearn

,Car_Name,Year,Present_Price,Driven_kms,Fuel_Type,Selling_type,Transmission,Owner,Car_Age
144,Hero Extreme,2014,0.787,15000,Petrol,Individual,Manual,0,11
209,elantra,2015,14.790,43535,Diesel,Dealer,Manual,0,10
227,i20,2012,6.790,52132,Diesel,Dealer,Manual,0,13
24,swift,2011,4.990,20000,Petrol,Dealer,Manual,0,14
278,brio,2015,5.900,14465,Petrol,Dealer,Manual,0,10
...,...,...,...,...,...,...,...,...,...
75,corolla altis,2013,14.680,72000,Petrol,Dealer,Manual,0,12
22,alto k10,2014,3.460,45280,Petrol,Dealer,Manual,0,11
72,etios liva,2011,5.710,43000,Petrol,Dealer,Manual,0,14
15,ertiga,2016,10.790,43000,Diesel,Dealer,Manual,0,9


In [34]:
pf.fit_transform(X_train_fe_sklearn[['Driven_kms','Year']])
sp = SplineTransformer(n_knots=3, degree=3)
sp.fit_transform(X_train_fe_sklearn[['Driven_kms']])
qt = QuantileTransformer()
qt.fit_transform(X_train_fe_sklearn[['Driven_kms']])

c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_data.py:2846: UserWarning: n_quantiles (1000) is greater than the total number of samples (224). n_quantiles is set to n_samples.
  warnings.warn(


array([[0.21524664],
       [0.69058296],
       [0.81165919],
       [0.3161435 ],
       [0.19282511],
       [0.28475336],
       [0.16591928],
       [0.01793722],
       [0.71973094],
       [0.44394619],
       [0.3632287 ],
       [0.84753363],
       [0.6367713 ],
       [0.48878924],
       [0.132287  ],
       [0.71973094],
       [0.92825112],
       [0.5044843 ],
       [0.65919283],
       [0.93946188],
       [0.11210762],
       [0.02690583],
       [0.83856502],
       [0.30941704],
       [0.85650224],
       [0.40358744],
       [0.73991031],
       [0.91479821],
       [0.15695067],
       [0.26681614],
       [0.21524664],
       [0.29147982],
       [0.37892377],
       [0.21524664],
       [0.92152466],
       [0.        ],
       [0.867713  ],
       [0.2735426 ],
       [0.01345291],
       [0.21524664],
       [0.0896861 ],
       [0.3632287 ],
       [0.95515695],
       [0.37892377],
       [0.1838565 ],
       [0.43049327],
       [0.30044843],
       [0.049

In [35]:
pf = PolynomialFeatures(degree=2)
qt = QuantileTransformer()
sp = SplineTransformer(n_knots=3, degree=3)

In [36]:
pf_pipeline = Pipeline(steps=[
    ('poly', pf),
    ('scale', StandardScaler())
])

#### 15. Преобразования для числовых и категориальных признаков

In [37]:
preprocessor_sklearn = ColumnTransformer(
    transformers=[
        ('num', s_scaler, num_features),  
        ('cat', l_encoder, cat_features), 
        ('quantile', qt, num_features),
        ('poly', pf_pipeline, ['Driven_kms', 'Year']),
        ('spline', sp, ['Driven_kms'])
    ],
    remainder='drop',
    )

In [38]:
X_train_fe_sklearn[['Driven_kms','Year']] = X_train_fe_sklearn[['Driven_kms','Year']].astype('float64')
X_train_fe_sklearn[['Driven_kms','Year']] = X_train_fe_sklearn[['Driven_kms','Year']].astype('float64')

In [39]:
X_train_sklearn_raw = preprocessor_sklearn.fit_transform(X_train_fe_sklearn, y_train)
X_train_fe_sklearn = pd.DataFrame(X_train_sklearn_raw, columns=preprocessor_sklearn.get_feature_names_out())

c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_data.py:2846: UserWarning: n_quantiles (1000) is greater than the total number of samples (224). n_quantiles is set to n_samples.
  warnings.warn(


In [40]:
with pd.option_context('display.max_rows', 5, 'display.max_columns', None):
    display (X_train_fe_sklearn)

,num__Year,num__Present_Price,num__Driven_kms,num__Owner,num__Car_Age,cat__Car_Name,cat__Fuel_Type,cat__Selling_type,cat__Transmission,quantile__Year,quantile__Present_Price,quantile__Driven_kms,quantile__Owner,quantile__Car_Age,poly__1,poly__Driven_kms,poly__Year,poly__Driven_kms^2,poly__Driven_kms Year,poly__Year^2,spline__Driven_kms_sp_0,spline__Driven_kms_sp_1,spline__Driven_kms_sp_2,spline__Driven_kms_sp_3,spline__Driven_kms_sp_4
0,0.115848,-0.787493,-0.551107,-0.19245,-0.115848,4.604134,3.274728,0.975781,3.886710,0.448430,0.114350,0.215247,0.0,0.551570,0.0,-0.551107,0.115848,-0.175831,-0.551880,0.115257,0.140503,0.663690,0.195779,2.823558e-05,0.0
1,0.466521,0.738642,0.125527,-0.19245,-0.466521,11.450000,9.798006,6.563671,3.886710,0.616592,0.899103,0.690583,0.0,0.385650,0.0,0.125527,0.466521,-0.078449,0.127270,0.466403,0.095385,0.640298,0.263502,8.151886e-04,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,0.817195,0.302697,0.112841,-0.19245,-0.817195,6.615101,10.219132,7.008628,4.315965,0.800448,0.773543,0.681614,0.0,0.199552,0.0,0.112841,0.817195,-0.081149,0.115048,0.817724,0.096126,0.640930,0.262159,7.846722e-04,0.0
223,-0.585500,-0.782807,-0.776375,-0.19245,0.585500,5.130944,3.423861,0.952256,4.315965,0.224215,0.156951,0.058296,0.0,0.775785,0.0,-0.776375,-0.585500,-0.187185,-0.777944,-0.586513,0.158194,0.666372,0.175433,8.542102e-07,0.0


In [41]:
pipeline_sklearn = Pipeline(steps=[
    ('transform', preprocessor_sklearn),
    ('model', regressor)
])

model_sklearn = pipeline_sklearn.fit(X_train, y_train)

c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_data.py:2846: UserWarning: n_quantiles (1000) is greater than the total number of samples (224). n_quantiles is set to n_samples.
  warnings.warn(


In [42]:
predictions = model_sklearn.predict(X_test) 
metrics = {}
metrics["MAE"] = mean_absolute_error(y_test, predictions)
metrics["MSE"] = mean_squared_error(y_test, predictions)
metrics["MAPE"] = mean_absolute_percentage_error(y_test, predictions)
print(metrics)

{'MAE': 0.5609600000000003, 'MSE': 1.4759497389333336, 'MAPE': 0.2149489962031561}


In [43]:
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

if experiment is None:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id

with mlflow.start_run(run_name="sklearn_regressor", experiment_id=experiment_id) as run:
   
    run_id = run.info.run_id 
    mlflow.sklearn.log_model(pipeline, 
                             artifact_path="models",
                             signature=signature,
                             input_example=input_example,
                             pip_requirements=req_file
                             )
    mlflow.log_metrics(metrics)

    mlflow.log_params(params_dict)

run = mlflow.get_run(run_id) 
assert (run.info.status =='FINISHED')

2025/11/05 19:13:26 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/05 19:13:26 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - numpy (current: 2.3.4, required: numpy==1.26.4)
 - mlflow (current: 3.5.1, required: mlflow==2.16)
 - seaborn (current: uninstalled, required: seaborn)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


2025/11/05 19:13:26 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - numpy (current: 2.3.4, required: numpy==1.26.4)
 - mlflow (current: 3.5.1, required: mlflow==2.16)
 - seaborn (current: uninstalled, required: seaborn)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


🏃 View run sklearn_regressor at: http://127.0.0.1:5000/#/experiments/818228273553268644/runs/2dbfaefb714d4c7daaa2d9e040693702
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/818228273553268644


#### 16. С использованием библиотеки mlxtend отобрали N наиболее важных признаков. Обучили модель и залогировали ее

In [44]:
class ColumnExtractor(object):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        return X[:, self.cols]
    
    def fit(self, X, y=None):
        return self

In [45]:
from mlxtend.feature_selection import SequentialFeatureSelector 
sfs = SequentialFeatureSelector(
    RandomForestRegressor(random_state=43), 
    k_features=3,
    forward=True,
    floating=False,
    scoring='neg_mean_absolute_error',
    cv=2
)

sfs.fit(X_train_fe_sklearn, y_train)

sfs_cols = list(X_train_fe_sklearn.columns[list(sfs.k_feature_idx_)])
sfs_idx = list(sfs.k_feature_idx_)

print(sfs_cols)

with open('sfs_cols.txt', 'w+') as f:
    f.write(str(sfs_cols))
with open('sfs_idx.txt', 'w+') as f:
    f.write(str(sfs_idx))


sfs_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor_sklearn),
    ('sfs_extractor', ColumnExtractor(sfs_idx)),
    ('model', regressor)
])

sfs_pipeline.fit(X_train, y_train)


predictions_sfs = sfs_pipeline.predict(X_test)
metrics = {}
metrics["MAE"] = mean_absolute_error(y_test, predictions_sfs)
metrics["MSE"] = mean_squared_error(y_test, predictions_sfs)
metrics["MAPE"] = mean_absolute_percentage_error(y_test, predictions_sfs)

print(metrics)


experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id
RUN_NAME = 'sequential_feature_selection'

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id 
    
   
    mlflow.sklearn.log_model(
        sfs_pipeline, 
        artifact_path="models",
        signature=signature,
        input_example=input_example,
        pip_requirements=req_file
    )
    mlflow.log_metrics(metrics)
    mlflow.log_artifact('sfs_cols.txt')
    mlflow.log_artifact('sfs_idx.txt')
    
    params = {
        'n_features_selected': len(sfs_cols),
        'forward': True,
        'floating': False,
        'scoring': 'neg_mean_absolute_error'
    }
    mlflow.log_params(params)
    
    model_version = mlflow.register_model(
        f"runs:/{run.info.run_id}/models",
        REGISTRY_MODEL_NAME,
        await_registration_for=300
    )
    
  
    client = mlflow.tracking.MlflowClient()
    client.update_model_version(
        name=REGISTRY_MODEL_NAME,
        version=model_version.version,
        description="Random Forest with Sequential Feature Selection"
    )

print(f"Model registered as version {model_version.version} of {REGISTRY_MODEL_NAME}")

c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_data.py:2846: UserWarning: n_quantiles (1000) is greater than the total number of samples (224). n_quantiles is set to n_samples.
  warnings.warn(
2025/11/05 19:13:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


['quantile__Year', 'quantile__Present_Price', 'quantile__Car_Age']
{'MAE': 0.48064928835978793, 'MSE': 0.9686046294707658, 'MAPE': 0.19588497145965655}


2025/11/05 19:13:35 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - numpy (current: 2.3.4, required: numpy==1.26.4)
 - mlflow (current: 3.5.1, required: mlflow==2.16)
 - seaborn (current: uninstalled, required: seaborn)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


2025/11/05 19:13:35 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - numpy (current: 2.3.4, required: numpy==1.26.4)
 - mlflow (current: 3.5.1, required: mlflow==2.16)
 - seaborn (current: uninstalled, required: seaborn)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
Successfully registered model 'car_price_prediction_model'.
2025/11/05 19:13:35 WARNING mlflow.tracking._model_registry.fluent: Run with id a184ad375e7a487fb98cd3f3f23483f9 has no artifacts at artifact path 'models', registering model based on models:/m-3d4be6706fe446de95024e5d2d8e30d5 instead
2025/11/05 19:13:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: car_price_prediction_model, version 1


🏃 View run sequential_feature_selection at: http://127.0.0.1:5000/#/experiments/818228273553268644/runs/a184ad375e7a487fb98cd3f3f23483f9
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/818228273553268644
Model registered as version 1 of car_price_prediction_model


Created version '1' of model 'car_price_prediction_model'.


#### 17. С помощью optuna настроили оптимальные параметры для модели, показывающей лучший результат. Обучили модель, залогировали ее

In [46]:
import optuna

def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'n_estimators': trial.suggest_int('n_estimators', 10, 100),
        'max_features': trial.suggest_float('max_features', 0.1, 1.0),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10)
    }
    
    regressor_trial = RandomForestRegressor(**params, random_state=44)
    pipeline_trial = Pipeline(steps=[
        ('preprocessor', preprocessor_sklearn), 
        ('sfs_extractor', ColumnExtractor(sfs_idx)),  
        ('model', regressor_trial)
    ])  
    
    pipeline_trial.fit(X_train, y_train)
    predictions = pipeline_trial.predict(X_test)
    mae = mean_absolute_error(y_test, predictions)

    return mae

study = optuna.create_study(direction='minimize')  
study.optimize(objective, n_trials=30)  

best_params = study.best_params
print("Лучшие параметры:", best_params)

regressor_optimized = RandomForestRegressor(**best_params, random_state=42)
sfs_optuna_pipeline = Pipeline(steps=[  
    ('preprocessor', preprocessor_sklearn), 
    ('sfs_extractor', ColumnExtractor(sfs_idx)),  
    ('model', regressor_optimized)
])

sfs_optuna_pipeline.fit(X_train, y_train)

predictions_optuna = sfs_optuna_pipeline.predict(X_test)
metrics = {
    "MAE": mean_absolute_error(y_test, predictions_optuna),
    "MSE": mean_squared_error(y_test, predictions_optuna),
    "MAPE": mean_absolute_percentage_error(y_test, predictions_optuna)
}

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id
RUN_NAME = 'sfs_feature_selection_optimised_Optuna'

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id 
    mlflow.sklearn.log_model(
        sfs_optuna_pipeline,  
        artifact_path="models",
        signature=signature,
        input_example=input_example,
        pip_requirements=req_file
    )
    mlflow.log_metrics(metrics)
    mlflow.log_params(best_params)
    mlflow.log_artifact('sfs_cols.txt')  
    mlflow.log_artifact('sfs_idx.txt')   
    
    model_version = mlflow.register_model(
        f"runs:/{run.info.run_id}/models",
        REGISTRY_MODEL_NAME,
        await_registration_for=300
    )
    
    client = mlflow.tracking.MlflowClient()
    client.update_model_version(
        name=REGISTRY_MODEL_NAME,
        version=model_version.version,
        description="Optimized Random Forest with Optuna and SFS feature selection"
    )

print(f"Model registered as version {model_version.version} of {REGISTRY_MODEL_NAME}")

[I 2025-11-05 19:13:36,004] A new study created in memory with name: no-name-f6d49f4d-0c04-481a-88eb-066d6d468402
c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_data.py:2846: UserWarning: n_quantiles (1000) is greater than the total number of samples (224). n_quantiles is set to n_samples.
  warnings.warn(
[I 2025-11-05 19:13:36,035] Trial 0 finished with value: 0.8255492420943374 and parameters: {'max_depth': 9, 'n_estimators': 34, 'max_features': 0.771913330755009, 'min_samples_split': 10, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.8255492420943374.
c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_data.py:2846: UserWarning: n_quantiles (1000) is greater than the total number of samples (224). n_quantiles is set to n_samples.
  warnings.warn(
[I 2025-11-05 19:13:36,060] Trial 1 finished with value: 0.579763549972723 and parameters: {'max_depth': 7, 'n_estimators': 14, 'max_feat

Лучшие параметры: {'max_depth': 7, 'n_estimators': 55, 'max_features': 0.9051612492110676, 'min_samples_split': 6, 'min_samples_leaf': 2}


2025/11/05 19:13:37 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - numpy (current: 2.3.4, required: numpy==1.26.4)
 - mlflow (current: 3.5.1, required: mlflow==2.16)
 - seaborn (current: uninstalled, required: seaborn)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


2025/11/05 19:13:37 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - numpy (current: 2.3.4, required: numpy==1.26.4)
 - mlflow (current: 3.5.1, required: mlflow==2.16)
 - seaborn (current: uninstalled, required: seaborn)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
Registered model 'car_price_prediction_model' already exists. Creating a new version of this model...
2025/11/05 19:13:38 WARNING mlflow.tracking._model_registry.fluent: Run with id 88e90dc8eb5c4e3fbd153e249ead3330 has no artifacts at artifact path 'models', registering model based on models:/m-5719511ac00c438eb7a7c368db8888c7 instead
2025/11/05 19:13:38 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: car_price_prediction_model,

🏃 View run sfs_feature_selection_optimised_Optuna at: http://127.0.0.1:5000/#/experiments/818228273553268644/runs/88e90dc8eb5c4e3fbd153e249ead3330
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/818228273553268644
Model registered as version 2 of car_price_prediction_model


#### 18. Проанализировали все прогоны и выбрали модель, показывающую наилучшее качество

In [47]:
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
runs = mlflow.search_runs(experiment_ids=[experiment.experiment_id])
best_run = runs.loc[runs['metrics.MAE'].idxmin()]
best_run_id = best_run['run_id']
best_mae = best_run['metrics.MAE']

print(f"Лучшая модель: Run ID {best_run_id}")
print(f"Лучший MAE: {best_mae}")
print(f"Метод: {best_run['tags.mlflow.runName']}")

Лучшая модель: Run ID a184ad375e7a487fb98cd3f3f23483f9
Лучший MAE: 0.48064928835978793
Метод: sequential_feature_selection


##### Вывод: Наилучшей моделью оказалась модель sequential_feature_selection

#### 19. Используем всю выборку для обучения модели sequential_feature_selection

In [48]:
X_full = df.drop('target', axis=1)
y_full = df['target']

print(f"Размер всей выборки: {X_full.shape}")

Размер всей выборки: (299, 9)


In [49]:
best_run = mlflow.get_run(best_run_id)
best_params = best_run.data.params

print("Параметры лучшей модели:", best_params)

final_pipeline = Pipeline(steps=[
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), num_features),  
            ('cat', TargetEncoder(), cat_features), 
            ('quantile', QuantileTransformer(), num_features),
            ('poly', Pipeline(steps=[
                ('poly', PolynomialFeatures(degree=2)),
                ('scale', StandardScaler())
            ]), ['Driven_kms', 'Year']),
            ('spline', SplineTransformer(n_knots=3), ['Driven_kms'])
        ],
        remainder='drop'
    )),
    ('sfs_extractor', ColumnExtractor(sfs_idx)),
    ('model', RandomForestRegressor(
        random_state=42,
        max_depth=int(best_params.get('model__max_depth', best_params.get('max_depth', 10))),
        n_estimators=int(best_params.get('model__n_estimators', best_params.get('n_estimators', 100))),
        max_features=float(best_params.get('model__max_features', best_params.get('max_features', 1.0))),
        min_samples_split=int(best_params.get('model__min_samples_split', best_params.get('min_samples_split', 2))),
        min_samples_leaf=int(best_params.get('model__min_samples_leaf', best_params.get('min_samples_leaf', 1)))
    ))
])

print("Обучаем модель на всей выборке...")
final_model = final_pipeline.fit(X_full, y_full)
print("Модель успешно обучена на всех данных!")

Параметры лучшей модели: {'floating': 'False', 'forward': 'True', 'n_features_selected': '3', 'scoring': 'neg_mean_absolute_error'}
Обучаем модель на всей выборке...
Модель успешно обучена на всех данных!


c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_data.py:2846: UserWarning: n_quantiles (1000) is greater than the total number of samples (299). n_quantiles is set to n_samples.
  warnings.warn(


In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np

final_predictions = final_model.predict(X_test)

cv_scores = cross_val_score(final_model, X_full, y_full, 
                           scoring='neg_mean_absolute_error', cv=5)
final_mae_cv = -np.mean(cv_scores)

metrics = {
    "MAE": mean_absolute_error(y_test, final_predictions),
    "MSE": mean_squared_error(y_test, final_predictions),
    "MAPE": mean_absolute_percentage_error(y_test, final_predictions)
}

print(f"CV MAE на всех данных: {final_mae_cv:.4f}")

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name='Final_Production_Full_Data', experiment_id=experiment_id) as run:
    mlflow.sklearn.log_model(
        final_model,
        artifact_path="models",
        signature=signature,
        input_example=input_example,
        pip_requirements=req_file
    )
    
    
    mlflow.log_metrics(metrics)
    
    
    mlflow.log_param("MAE_cv", final_mae_cv)
    mlflow.log_param("MAE_cv_std", np.std(cv_scores))
    
    mlflow.log_params(best_params)
    mlflow.log_param("training_data", "full_dataset")
    mlflow.log_param("training_samples", len(X_full))
    mlflow.log_param("original_best_run", best_run_id)
    mlflow.log_param("model_type", "sequential_feature_selection")
    
    production_version = mlflow.register_model(
        f"runs:/{run.info.run_id}/models",  
        REGISTRY_MODEL_NAME,
        await_registration_for=300
    )
    
    client = mlflow.tracking.MlflowClient()  
    client.set_model_version_tag(
        name=REGISTRY_MODEL_NAME,
        version=production_version.version,
        key="stage",
        value="Production"
    )
    
    client.update_model_version(
        name=REGISTRY_MODEL_NAME,
        version=production_version.version,
        description=f"Final Production Model - Sequential Feature Selection trained on full dataset ({len(X_full)} samples)"
    )
    
    production_run_id = run.info.run_id
    print(f"Production модель зарегистрирована как версия {production_version.version}")

c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_data.py:2846: UserWarning: n_quantiles (1000) is greater than the total number of samples (239). n_quantiles is set to n_samples.
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_data.py:2846: UserWarning: n_quantiles (1000) is greater than the total number of samples (239). n_quantiles is set to n_samples.
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_data.py:2846: UserWarning: n_quantiles (1000) is greater than the total number of samples (239). n_quantiles is set to n_samples.
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_data.py:2846: UserWarning: n_quantiles (1000) is greater than the total number of samples (239). n_quantiles is set to n_samples.
  warnings.warn(
c:\Users\Lenovo\AppData\

CV MAE на всех данных: 0.9057


2025/11/05 19:27:49 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - numpy (current: 2.3.4, required: numpy==1.26.4)
 - mlflow (current: 3.5.1, required: mlflow==2.16)
 - seaborn (current: uninstalled, required: seaborn)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


2025/11/05 19:27:49 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - numpy (current: 2.3.4, required: numpy==1.26.4)
 - mlflow (current: 3.5.1, required: mlflow==2.16)
 - seaborn (current: uninstalled, required: seaborn)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
Registered model 'car_price_prediction_model' already exists. Creating a new version of this model...
2025/11/05 19:27:50 WARNING mlflow.tracking._model_registry.fluent: Run with id a0c1e800cf7c4171847779c1deb7ca27 has no artifacts at artifact path 'models', registering model based on models:/m-7a281a5cbf324c9eb6c8c8cdb37826eb instead
2025/11/05 19:27:50 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: car_price_prediction_model,

Production модель зарегистрирована как версия 4
🏃 View run Final_Production_Full_Data at: http://127.0.0.1:5000/#/experiments/818228273553268644/runs/a0c1e800cf7c4171847779c1deb7ca27
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/818228273553268644


#### 20. Значения метрик для полученной модели

In [60]:
print(f"MAE для модели обученной на всей выборке: {metrics['MAE']:.2%}")
print(f"MSE для модели обученной на всей выборке: {metrics['MSE']:.2%}")
print(f"MAPE для модели обученной на всей выборке: {metrics['MAPE']:.2%}")

MAE для модели обученной на всей выборке: 25.62%
MSE для модели обученной на всей выборке: 42.49%
MAPE для модели обученной на всей выборке: 8.87%
